In [55]:
!nvidia-smi

Fri Nov 17 20:14:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:C1:00.0 Off |                  N/A |
|  0%   51C    P2   128W / 370W |  17210MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# seed

In [56]:
import random
import numpy as np
import torch


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed = 11032006
set_seed(seed)


In [57]:
import torch

print(torch.cuda.is_available())

True


# main

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from GoDataset import GoDataset
from parmFinder import ParmFinder
from Trainer import Trainer

config = {
    "input_dim": 19 * 19 * 2,
    "num_heads": 2,
    "ffn_dim": 512,
    "num_layers": 3,
    "depthwise_conv_kernel_size": 3,
    "dropout": 0.1,
    "use_group_norm": False,
    "convolution_first": False,
    "lr": 0.00001,
    "gen_path": "data/models/gen",
    "dis_path": "data/models/dis",
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    # "device": torch.device("cpu"),
    "batch_size": 16,
    "clip_value": 1,
    "data_len": 64,
    "epochs": 20,
    "early_stop": 400,
    "selected": 0
}

goDataset = GoDataset("data/train/dan_train.csv", config["data_len"])
train_len = int(0.8 * len(goDataset))
val_len = len(goDataset) - train_len
train_dataset, val_dataset = torch.utils.data.random_split(
    goDataset, [train_len, val_len]
)
train_loader = DataLoader(
    train_dataset,
    batch_size=int(config["batch_size"]),
    shuffle=True,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset, batch_size=int(config["batch_size"]), shuffle=False, pin_memory=True
)
trainer = Trainer(config, train_loader, val_loader)
statistic = trainer.train()


['/home/alanchou/Go', '/home/alanchou/micromamba/envs/.venv/lib/python39.zip', '/home/alanchou/micromamba/envs/.venv/lib/python3.9', '/home/alanchou/micromamba/envs/.venv/lib/python3.9/lib-dynload', '', '/home/alanchou/micromamba/envs/.venv/lib/python3.9/site-packages']


TypeError: 'module' object is not callable

In [ ]:

for key, value in statistic.items():
    plt.plot(value, label=key)
    plt.legend()
    plt.savefig(f"plot/{key}.png")
    plt.clf()